In [ ]:
# Import libraries
import warnings
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
# Defaults
plt.rcParams['figure.figsize'] = (20.0, 10.0)
plt.rcParams.update({'font.size': 12})
plt.style.use('ggplot')

In [ ]:
# Load data
data = pd.read_csv('Monthly-car-sales.csv', engine='python', skipfooter=3)
# pre-processing
data['Month']=pd.to_datetime(data['Month'], format='%Y-%m-%d')
data.set_index(['Month'], inplace=True)

# Plot data
data.plot()
plt.title('Monthly Car Sales')
plt.ylabel('Sales')
plt.xlabel('Date')
plt.show()

In [ ]:
# Define the d and q parameters to take any value between 0 and 1
q = d = range(0, 2)
# Define the p parameters to take any value between 0 and 3
p = range(0, 4)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Grid Search / Hyperparameter Optimization for Model selection - be able to pick up the smallest AIC Value. 
# Generate all different combinations of seasonal p, q and q
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

# SARIMAX returns the AIC Value
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
# Train the data and predict the last year
training_data = data['1960-01-01':'1967-12-01']
test_data = data['1968-01-01':'1968-12-01']

In [ ]:
data

In [ ]:
# Ignore warning messages
warnings.filterwarnings("ignore")

# Generate an AIC Value
AIC = []
SARIMAX_model = []
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(training_data,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('SARIMAX{}x{} - AIC:{}'.format(param, param_seasonal, results.aic), end='\r')
            AIC.append(results.aic)
            SARIMAX_model.append([param, param_seasonal])
        except:
            continue
results

In [ ]:
mod = sm.tsa.statespace.SARIMAX(training_data,
                                order=param,
                                seasonal_order=param_seasonal,
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

print('SARIMAX{}x{} - AIC:{}'.format(param, param_seasonal, results.aic), end='\r')
AIC.append(results.aic)
SARIMAX_model.append([param, param_seasonal])

In [ ]:
# AIC Value?
mod

In [ ]:
print('The smallest AIC is {} for model SARIMAX{}x{}'.format(min(AIC), SARIMAX_model[AIC.index(min(AIC))][0],SARIMAX_model[AIC.index(min(AIC))][1]))

In [ ]:
# Fit the model
mod = sm.tsa.statespace.SARIMAX(training_data,
                                order=SARIMAX_model[AIC.index(min(AIC))][0],
                                seasonal_order=SARIMAX_model[AIC.index(min(AIC))][1],
                                enforce_stationarity=False,
                                enforce_invertibility=False)

results = mod.fit()

In [ ]:
# Use plot diagnostics to check if it meets our expecatations
results.plot_diagnostics(figsize=(20, 14))
plt.show()

In [ ]:
# Results (TRIAL): 3 types of forecasting

In [ ]:
# Sample prediction with 1-step ahead forecasting of the last year 1967 
# Model is asked to predict data using existing data 

pred0 = results.get_prediction(start='1966-01-01', dynamic=False)
pred0_ci = pred0.conf_int()

In [ ]:
# Sample prediction with dynamic forecasting of the last year 1967
# Model is asked to predict data using existing data 

pred1 = results.get_prediction(start='1966-01-01', dynamic=True)
pred1_ci = pred1.conf_int()

In [ ]:
# True forecasting 
# Model is asked to predict data its not seen before 

pred2 = results.get_forecast('1970-12-01')
pred2_ci = pred2.conf_int()
print(pred2.predicted_mean['1968-01-01':'1968-12-01'])

In [ ]:
# Plot all data

ax = data.plot(figsize=(25, 16))
pred0.predicted_mean.plot(ax=ax, label='1-step-ahead Forecast (get_predictions, dynamic=False)')
pred1.predicted_mean.plot(ax=ax, label='Dynamic Forecast (get_predictions, dynamic=True)')
pred2.predicted_mean.plot(ax=ax, label='Dynamic Forecast (get_forecast)')
ax.fill_between(pred2_ci.index, pred2_ci.iloc[:, 0], pred2_ci.iloc[:, 1], color='k', alpha=.1)
plt.title('Monthly Car Sales')
plt.ylabel('Sales')
plt.xlabel('Date')
plt.legend()
plt.show()